# Mapping Part 2 — Workbook

In this lesson, we're going to learn how to analyze and visualize geographic data.

*Note: You can explore this [workbook](https://mybinder.org/v2/gh/INFO1350/Intro-CA-SP21/master?urlpath=lab/tree/book/07-Mapping/02.5-Mapping-WORKBOOK.ipynb) in the cloud via Binder.*

## Torn Apart / Separados

### Read in Data

The data in this section was drawn from [Torn Apart / Separados Project](https://github.com/xpmethod/torn-apart-open-data). It maps the locations of Immigration and Customs Enforcement (ICE) detention facilities, as featured in [Volume 1](http://xpmethod.plaintext.in/torn-apart/volume/1/).

In [ ]:
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

You can find this data on GitHub: https://github.com/xpmethod/torn-apart-open-data/blob/master/iceFacs.csv

In [ ]:
fac_df = pd.read_csv("../data/ICE-facilities.csv")
# Drop any blank lat, lon coordinates
fac_df = fac_df.dropna(subset=['lat', 'lon'])
fac_df

### Import Folium

In [ ]:
!pip install folium

In [ ]:
import folium

Let's establish a base map

In [ ]:
US_map = folium.Map(location=[42, -102], zoom_start=4)
US_map

### Add a Circle Marker

There are a few [different kinds of markers](https://python-visualization.github.io/folium/quickstart.html#Markers) that we can add to a Folium map, including circles. To make a circle, we can call `folium.CircleMarker()` with a particular radius and the option to fill in the circle.

You can explore more customization options in the [Folium documentation](https://python-visualization.github.io/folium/modules.html#folium.vector_layers.CircleMarker). We're also going to add a hover `tooltip`.

In [ ]:
# Establish base map
US_map = folium.Map(location=[42, -102], zoom_start=4)

# Loop through rows in the Pandas DataFrame
for index, row in fac_df.iterrows():
    
    # Make a caption
    caption_info = f"""{row['Name']} <br>
                    {row['City']}, {row['State']}""" 
    
    folium.CircleMarker(
                # Latitude, longitude for each marker
                location=[row['lat'], row['lon']],
        
                # Size, fill, color of the circle marker
                radius = 10, fill = True, color='orange',
        
                # Text that goes into the popup or tooltip
                tooltip = caption_info 
                  
                 ).add_to(US_map)

US_map

## Your Turn!

- What is this map showing us about ICE (U.S Immigration and Customs Enforcement) that words alone or even a DataFrame couldn't capture?
- What other information might be important to have in the popup or tooltip?
- What other design choices would you make to improve this map? What consequences could these decisions have in the world? 

Pick at least one other category from the DataFrame and include it in the popup or toolip. You can review the existing columns in the DataFrame below.

In [ ]:
fac_df.columns

In [ ]:
fac_df.sample(5)

Add another category to the tooltip/popup

In [ ]:
# Establish base map
US_map = folium.Map(location=[42, -102], zoom_start=4)

# Loop through rows in the Pandas DataFrame
for index, row in fac_df.iterrows():
    
    # Make a caption -- <br> is HTML for a line break
    caption_info = f"""{row['Name']} <br>
                    {row['City']}, {row['State']} <br>
                    More info here: """ 
                    # ☝️ Add another category into the caption above and/or alter the design! ☝️

    folium.CircleMarker(
                # Latitude, longitude for each marker
                location=[row['lat'], row['lon']],
        
                # Size, fill, color of the circle marker
                radius = 10, fill = True, color='orange',
        
                # Text that goes into the popup or tooltip
                tooltip = caption_info 
                  
                 ).add_to(US_map)

US_map

Save the map as an HTML file

In [ ]:
US_map.save("ICE-map.html")

## Choropleth Maps

Choropleth map = a map where areas are shaded according to a value

The data in this section was drawn from [Torn Apart / Separados Project](https://github.com/xpmethod/torn-apart-open-data). This data maps the "cumulative ICE awards since 2014 to contractors by congressional district," as featured in [Volume 2](http://xpmethod.plaintext.in/torn-apart/volume/2/).

To create a chropleth map with Folium, we need to pair a "geo.json" file (which indicates which parts of the map to shade) with a CSV file (which includes the variable that we want to shade by).

The following data was drawn from [the Torn Apart / Separados project](https://github.com/xpmethod/torn-apart/tree/master/data/districts)

First, we will examine the geoJSON file. To learn more about geoJSON, you can explore here: https://geojson.io/#map=13/42.4349/-76.5028

In [ ]:
geojson_df = pd.read_json("../data/ICE_money_districts.geo.json")

In [ ]:
geojson_df['features'][0]

In [ ]:
districts_df = pd.read_csv("../data/ICE_money_districts.csv")
districts_df = districts_df.dropna(subset=['districtName', 'representative'])
districts_df

## Your Turn!

Run the following cells and inspect the maps. Then answer the discussion questions at the bottom.

Here we are creating a choropleth map by pairing geoJSON data and a Pandas DataFrame.

In [ ]:
US_map = folium.Map(location=[42, -102], zoom_start=4)

choro_map = folium.Choropleth(
    # Geojson data
    geo_data = "../data/ICE_money_districts.geo.json",
    # DataFrame data
    data = districts_df,
    # Columns to include from DataFrame
    columns = ['districtName', 'total_awards'],
    # Shared category from geojson
    key_on = 'feature.properties.districtName',
    # Color palette
    fill_color = 'GnBu',
    line_opacity = 0.2,
    legend_name= 'Total ICE Money Received'
).add_to(US_map)

US_map

### Add a Tooltip to Choropleth

In [ ]:
tooltip = folium.features.GeoJson(
    "../data/ICE_money_districts.geo.json",
    tooltip=folium.features.GeoJsonTooltip(
        fields=['representative',
                'state',
                'party', 
                'total_value', 
                'profiteer'],
                localize=True))
US_map.add_child(tooltip)
US_map

## Discussion

To see how our map compares to the Torn Apart / Separados map, you can explore here: http://xpmethod.columbia.edu/torn-apart/volume/2/

- What does this map (or the Torn Apart/Separados map) show us about ICE that is different from the previous map?
- How do the design and tooltip choices of this map influence the argument being made?